<h1 style="color:SteelBlue; font-size:200%; line-height:1.5">Определение возраста человека по его фотографии</h1>

**Задача:** Построить модель, которая по фотографии определит приблизительный возраст человека.

**Цель:** Определять возраст покупателей в прикассовой зоне, чтобы делать рекомендации товаров и контролировать продажу алкоголя несовершеннолетним.

<h1 style="color:SteelBlue; font-size:200%; line-height:1.5">1. Загрузка данных, исследовательский анализ</h1>

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras import Sequential, regularizers, callbacks, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [ ]:
def load_train(path):
    train_datagen = ImageDataGenerator(
        rescale=1/255.,
        validation_split = 0.25,
        horizontal_flip = True
    )
    labels = pd.read_csv(path + 'labels.csv')
    train_flow = train_datagen.flow_from_dataframe(
        labels, 
        directory = path + 'final_files/final_files/',
        x_col="file_name",
        y_col="real_age",
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=12345)

    return train_flow

def load_test(path):
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255
        )

    labels = pd.read_csv(path + 'labels.csv')
    test_flow = test_datagen.flow_from_dataframe(
        labels, 
        directory = path + 'final_files/final_files/',
        x_col="file_name",
        y_col="real_age",
        target_size=(224, 224),
        batch_size=100,
        class_mode='raw',
        subset='validation',
        seed=12345)

    return test_flow

In [ ]:
path = '/kaggle/input/appa-real-face-cropped/'
train_flow = load_train(path)
test_flow = load_test(path)
labels = pd.read_csv(path + 'labels.csv')

In [ ]:
display(labels.head()) 
display(labels.info())

In [ ]:
print('Размер выборки:', labels.shape)
print('пропущенных значений:')
print(labels.isna().sum())

Размер фотографий в наборе данных:

In [ ]:
im_size_x, im_size_y = np.array([]), np.array([])
for i in range(len(labels)):
    im = Image.open(path + 'final_files/final_files/' + labels.iloc[i, 0])
    im_size_x = np.append(im_size_x, im.size[0])
    im_size_y = np.append(im_size_y, im.size[1])

In [ ]:
print('Неквадратных изображений:', np.sum( (im_size_x - im_size_y) != 0) )
not_rect = \
    ((abs(im_size_x/im_size_y) > 1.1) 
     + 
     (abs(im_size_x/im_size_y) < 0.9))

print( 'Изображения с разницей ширины и высоты более 10%:', np.sum(not_rect))
print( 'Доля таких изображений', np.sum(not_rect) / len(labels) )

В основном набор данных представлен квдратными изображениями. Но небольшая доля изображений будет растянута или сжата на более чем 10% при предобработке (resizing) данных в keras.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(16,4))
ax1, ax2 = axs.flatten()
ax1.hist(im_size_x, bins=100)
ax1.set_xlabel('x, px')
ax2.set_xlabel('x, px')
sns.boxplot(im_size_x, whis = [5,95])
plt.suptitle('Размеры изображений в выборке, px')
plt.show()

In [ ]:
print('Минимальный размер изображений по x:', im_size_x.min())
print('Средний размер изображений по x:', im_size_x.mean().round(0))
print('Медианный размер изображений по x:', np.median(im_size_x))
print('Максимальный размер изображений по x:', np.max(im_size_x))

In [ ]:
pd.Series(im_size_x).describe()

Размеры изображений в выборке имеют существенный разброс. 75% изображений больше 220 px по оси x. Нужно выполнять resize. Причём, растягивать большое количество изображений, наверное, нежелательно, т.к. на растянутых изображениях изначально группы пикселей несут меньше информации, чем на сжатых. Если использовать размер 224x224, растянутыми более чем в два раза окажутся:

In [ ]:
print( round( 100*np.sum(im_size_x < 112) / len(labels), 1) , '% изображений')

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(16,4))
ax1, ax2 = axs.flatten()
ax1.hist(labels.real_age, bins=50)
ax1.set_xlabel('возраст, лет')
sns.boxplot(labels.real_age, whis = [5,95])
ax2.set_xlabel('возраст, лет')
plt.suptitle('Возраст людей в выборке, лет')
plt.show()

In [ ]:
labels.real_age.describe()

Люди в выборке по возрасту распределены неравномерно. Но, на мой взгляд, выборка в целом соответствует людям, посещающим магазины. Однако виден провал в "тинейджерском интервале" и с контролем продажи алкоголя подросткам могут быть проблемы:

In [ ]:
print('Людей старше 13 и младше 18 лет в выборке:', sum ((labels.real_age > 13).values * (labels.real_age < 18).values))

Этого мало для хорошего обучения, на мой взгляд.

In [ ]:
X, y = next(train_flow)

In [ ]:
big_img_index = np.argsort(im_size_x)[-1]
small_img_index = np.argsort(im_size_x)[0]
img_big = Image.open(path + 'final_files/final_files/' + labels.iloc[big_img_index, 0])
img_small = Image.open(path + 'final_files/final_files/' + labels.iloc[small_img_index, 0])
plt.imshow(img_big)
plt.title('Самое большое и самое маленькое фото:')
plt.show()
plt.imshow(img_small)
plt.show()

In [ ]:
# выводим 12 изображений
fig = plt.figure(figsize=(10,10))
for i in range(12):
    fig.add_subplot(3, 4, i+1)
    plt.imshow(X[i])
    # для компактности удаляем оси и прижимаем изображения друг к другу
    plt.title( 'Age: ' + str(y[i]) )
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

<h1 style="color:SteelBlue; font-size:200%; line-height:1.5">Выводы по исследовательскому анализу</h1>

В наборе данных 7591 изображений лиц, с разбросом по размерам, часть фото - неквадратные. Выборка по возрасту в целом соответствует людям, посещающим магазины, однако несбалансирована. Есть провал в "тинейджерском интервале" и с контролем продажи алкоголя подросткам могут быть проблемы.

<h1 style="color:SteelBlue; font-size:200%; line-height:1.5">2. Обучение модели</h1>

Обучение модели с архитектурой, построенной на базе [статьи 1](https://github.com/emredogan7/age-estimation-by-CNN/blob/master/doc/report.pdf) и [статьи 2](https://talhassner.github.io/home/publication/2015_CVPR).

In [ ]:
def create_model(input_shape):
    model = Sequential()
    optimizer = Adam(lr=0.001)
    # 224x224x3 ->
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(7, 7), 
                     padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(BatchNormalization(momentum=0.9))

    model.add(Conv2D(filters=256, kernel_size=(5, 5), 
                     padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(BatchNormalization(momentum=0.9))

    model.add(Conv2D(filters=384, kernel_size=(3, 3), 
                     padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())

    model.add(Dropout(0.2))

    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=16, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=1, activity_regularizer = regularizers.l2(0.01), activation='relu'))

    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

    return model


def train_model(model, train_flow, test_flow, batch_size=None, epochs=10,
                steps_per_epoch=None, validation_steps=None, callbacks_list = None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_flow)
    if validation_steps is None:
        validation_steps = len(test_flow)

    model.fit(train_flow,
              validation_data=test_flow,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps, 
              epochs=epochs,
              callbacks = callbacks_list,
              verbose=2,
              shuffle=True)

    return model

In [ ]:
model = create_model((224, 224, 3))
model.summary()

In [ ]:
callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath='cv_arbf_model_{epoch}.h5',
        # Путь по которому нужно сохранить модель
        # Два параметра ниже означают, что мы перезапишем
        # текущий чекпоинт в том и только в том случае, когда
        # улучшится значение `val_loss`.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

In [ ]:
model = train_model(model, train_flow, train_flow, epochs=50, steps_per_epoch=None, validation_steps=None, callbacks_list = callbacks_list)

In [ ]:
model = models.load_model('./cv_arbf_model_45.h5')
model.summary()

На своей модели получилось достичь качества на тестовой выборке MAE = 5.8542, но нестабильно. Модель недоучилась, для стабильного качества нужно больше эпох.